In [4]:
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader
from collections import Counter
from itertools import chain
from torch.nn.utils.rnn import pad_sequence
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

In [5]:
import os
import pandas as pd

# Определяем путь к папке
base_path = '/Users/osintsev/Study/YandexML/3/ml_trainings.alien_translation'

# Полные пути к файлам
train_path = os.path.join(base_path, 'train')
val_path = os.path.join(base_path, 'val')
test_path = os.path.join(base_path, 'test_no_reference')

# Загрузка данных
train_data = pd.read_json(train_path, lines=True)
val_data = pd.read_json(val_path, lines=True)
test_data = pd.read_json(test_path, lines=True)

# Проверка загруженных данных
print(train_data.head())
print(val_data.head())
print(test_data.head())

                                                 dst  \
0                                      - Intriguing.   
1  He would need to repeat his vows in the land o...   
2                 You couldn't even answer my texts?   
3                                How fast do you go?   
4  He's talking about a few right here in Lisbon,...   

                                                 src  
0                                  ◄▴◓◠▨ ◨▽◠▦◈◬◓▪▼◬▵  
1  ▽◪◎◗▦◫▦◫ ▫▴▨◓◠◓ ▴▫◎◪▱◫ ◚▴ ◞◧▦◞▾▢▱◨▨ ◒◠◓◠◀▪▦◈◠▦...  
2               ◄▴◞◠▸▱◠◓▪◎◠ ◀◫▱◪ ▼◪◚◠▻ ◚▴◓▴◎◪◈◗▦ ◎◫?  
3                    ▯◪ ▨◠◈◠◓ ◞◭◓◠▫ ◳◠▻◬◳◧◓ ◞▴▦◗▦▨◫?  
4  ◈◠ ◧▱◠▦ ◀◫◓ ▨◠◉ ◂▱◠▽◈◠▦ ◀◠▷◞◪◈◗◳◧◓■ ◉◧◐▾▦▱◨◐▾ ...  
                                                 dst  \
0  The hosts regrouped, and Bouchard evened the s...   
1  A new cancer vaccine may teach the immune syst...   
2  Currently, language subjects are as popular as...   
3  It's no surprise that restaurant chains contin...   
4  Sales of drinks in pubs and bars increased by ... 

In [1]:
def simple_tokenizer(text):
    return text.split()

def build_vocab(data, tokenizer, specials=["<pad>", "<unk>", "<sos>", "<eos>"]):
    counter = Counter(chain(*[tokenizer(text) for text in data]))
    vocab = {token: idx for idx, token in enumerate(specials)}
    for token in counter.keys():
        if token not in vocab:
            vocab[token] = len(vocab)
    return vocab

In [9]:
vocab_src = build_vocab(train_data['src'].to_list(), simple_tokenizer)
vocab_dst = build_vocab(train_data['dst'].to_list(), simple_tokenizer)


def tokenize(text, vocab, tokenizer):
    return [vocab.get("<sos>")] + [vocab.get(token, vocab.get("<unk>")) for token in text] + [vocab.get("<eos>")]

In [11]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, vocab_src, vocab_dst, tokenizer_src, mode='train'):
        self.vocab_src = vocab_src
        self.vocab_dst = vocab_dst
        self.tokenizer_src = tokenizer_src
        self.mode = mode
        self.df = df

    def __len__(self): 
        return len(self.df) 

    def __getitem__(self, index):
        src_text = self.df.loc[index]['src']
        src = tokenize(src_text, self.vocab_src, self.tokenizer_src)

        if mode == 'train':
            dst_text = self.df.loc[index]['dst']
            dst = tokenize(dst_text, self.vocab_src, self.tokenizer_src)
            return torch.tensor(src), torch.tensor(dst)
        else:
            return torch.tensor(src)


In [ ]:
def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    src_batch = pad_sequence(src_batch, batch_first=False, padding_value=vocab_src["<pad>"])
    trg_batch = pad_sequence(trg_batch, batch_first=False, padding_value=vocab_dst["<pad>"])
    return src_batch, trg_batch 


train_dataset = CustomData(train_data, )